# Survival kit Torch Distribution

In [2]:
import torch
import gpytorch
import torch.distributions as dist
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions import Normal, Independent

from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood, FixedNoiseGaussianLikelihood
from gpytorch.models import ExactGP
from gpytorch.mlls import ExactMarginalLogLikelihood

Article

https://bochang.me/blog/posts/pytorch-distributions/

Doc Torch

https://docs.pytorch.org/docs/1.3.0/distributions.html#multivariatenormal

**Paramètres de la distribution:**

event_shape : dimension de la variable

batch_shape : batch size, indépendent, non identiques

**Arguments de rsample:**

sample_shape : nombre d'échantillons iid

In [3]:
# Crée une distribution MVN
# - covariance_matrix est de dimension 2x2, donc la dimension de la variable aléatoire est 2 : event_shape = 2
# - loc est le vecteur de moyenne de dimension 2

mvn = MultivariateNormal(
    loc=torch.zeros(2),
    covariance_matrix=torch.tensor([[1.0, 0.5], [0.5, 1.0]])
)

samples = mvn.rsample((1000,))  # Génère 1000 échantillons

print(samples.shape)  # Affiche la forme des échantillons

torch.Size([1000, 2])


In [4]:
# crée une autre distribution MVN
# - covariance_matrix toujours 2x2 => event_shape = 2
# - loc déduit du tensor en input : shape (5, 3, 2) + event_shape = 2 => batch_shape = (5, 3)

mvn = MultivariateNormal(
    loc=torch.randn(5,3,2),
    covariance_matrix=torch.eye(2)
)

samples = mvn.rsample((10,))  # Génère 10 échantillons

print(samples.shape)  # Affiche la forme des échantillons

torch.Size([10, 5, 3, 2])


In [5]:
# crée une autre distribution MVN
# - covariance_matrix toujours 2x2 => event_shape = 2
# - loc déduit du tensor en input : shape (5, 3, 2) + event_shape = 2 => batch_shape = (5, 3)

try:
    mvn = MultivariateNormal(
        loc=torch.randn(5,3,4),
        covariance_matrix=torch.eye(2)
    )

    samples = mvn.rsample((10,))  # Génère 10 échantillons
except RuntimeError as e:
    print(f"Non correspondance entre la taille de la matrice de covariance et la dernière dimension de loc")

Non correspondance entre la taille de la matrice de covariance et la dernière dimension de loc


### Independent class

The Independent class does not represent any probability distribution. Instead, it creates a new distribution instance by “reinterpreting” some of the batch shapes of an existing distribution as event shapes.

torch.distributions.independent.Independent(base_distribution,
reinterpreted_batch_ndims, validate_args=None)

In [6]:

normal = Normal(
    loc = torch.randn(5,3,2),
    scale = torch.ones(5,3,2)
)

print(f"AVANT Independent:")
print(f"Caractéristiques de la distribution normale : batch_shape = {normal.batch_shape}, event_shape = {normal.event_shape}")

# use Independent pour recaster la dernière dimension de 'batch_shape' dans 'event_shape' 
independent_normal_1 = Independent(normal, 1) 
print(f"APRES Independent(normal, 1):")
print(f"Caractéristiques de la distribution normale indépendante : batch_shape = {independent_normal_1.batch_shape}, event_shape = {independent_normal_1.event_shape}")

# use Independent pour recaster les deux dernières dimensions de 'batch_shape' dans 'event_shape' 
independent_normal_2 = Independent(normal, 2) 
print(f"APRES Independent(normal, 2):")
print(f"Caractéristiques de la distribution normale indépendante : batch_shape = {independent_normal_2.batch_shape}, event_shape = {independent_normal_2.event_shape}")

AVANT Independent:
Caractéristiques de la distribution normale : batch_shape = torch.Size([5, 3, 2]), event_shape = torch.Size([])
APRES Independent(normal, 1):
Caractéristiques de la distribution normale indépendante : batch_shape = torch.Size([5, 3]), event_shape = torch.Size([2])
APRES Independent(normal, 2):
Caractéristiques de la distribution normale indépendante : batch_shape = torch.Size([5]), event_shape = torch.Size([3, 2])


# For GP-VAE

In [7]:
T = 50 # sequence length
D_z = 8 # z dimension
B = 16 # batch size

mvn = MultivariateNormal(
    loc=torch.zeros(T, B, D_z),
    covariance_matrix=torch.eye(D_z)
)

print(f"Caractéristiques de la distribution MVN : batch_shape = {mvn.batch_shape}, event_shape = {mvn.event_shape}")

# sampling one sample
print(f"Génère 1 échantillon")
sample = mvn.rsample((1,))  # Génère un échantillon
print(f"Shape de l'échantillon : {sample.shape}")  # Affiche la forme de l'échantillon

# sampling K samples
K = 10
print(f"Génère {K} échantillons")
samples = mvn.rsample((K,))  # Génère K échantillons
print(f"Shape des échantillons : {samples.shape}")  # Affiche la forme des échantillons

Caractéristiques de la distribution MVN : batch_shape = torch.Size([50, 16]), event_shape = torch.Size([8])
Génère 1 échantillon
Shape de l'échantillon : torch.Size([1, 50, 16, 8])
Génère 10 échantillons
Shape des échantillons : torch.Size([10, 50, 16, 8])


### Processus Gaussien avec GPyTorch

Use case :

- y_train : on a échantilloné **un** sample $z_{1:T}$ de dimension $D_z$ : tensor shape $B \times T \times D_z$
- x_train : instants $(t_i)_{i=1,...T}$ : tensor shape $B \times T \times 1$

In [33]:
T = 50 # sequence length
D_z = 8 # z dimension
B = 16 # batch size

print(f"Batch size = {B}, sequence length = {T}, z dimension = {D_z}")

Batch size = 16, sequence length = 50, z dimension = 8


In [68]:
# TARGETS BRUTES
# ---------------

# train_x est un batch de B séquences temporelles de longueur T
train_x = torch.randn((B, T, 1))  # B x T x 1
# la target est un sample d'un batch de B séquences temporelles de longueur T de variables de dimension D_z
train_y = torch.randn((B, T, D_z))   # B x T x D_z

# RESHAPES
# --------

# duplicate train_x along the last dimension to match the output dimension D_z
train_x = train_x.expand(-1, -1, D_z)  # B x T x D_z
# permute les deux dernières dimensions pour avoir la forme (B, D_z, T)
train_x = train_x.permute(0, 2, 1)  # B x D_z x T
train_y = train_y.permute(0, 2, 1)  # B x D_z x T

# batch augmenté en B x D_z
train_x = train_x.reshape(B * D_z, T, 1)  # (B * D_z) x T
train_y = train_y.reshape(B * D_z, T)  # (B * D_z) x T
# batch_shape = (B * D_z)
bs = B * D_z  # B x D_z

# REPORT
# ------

print(f"train_x shape: {train_x.shape}")
print(f"train_y shape: {train_y.shape}")
print(f"batch_shape: {bs}")

train_x shape: torch.Size([128, 50, 1])
train_y shape: torch.Size([128, 50])
batch_shape: 128


In [ ]:
# We will use the simplest form of GP model, exact inference

#--------------------------------------------
#   BATCH VERSION
#--------------------------------------------

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        
        """An __init__ method that takes the training data and a likelihood, 
        and constructs whatever objects are necessary for the model"s forward method. 
        This will most commonly include things like a mean module and a kernel module.
        
        Inputs:
        - train_x: the training data, a tensor of shape (n, d) where n is the number of training points and d is the input dimension.
        - train_y: the training targets, a tensor of shape (n,) or (n, 1) where n is the number of training points.
        - likelihood: a likelihood object, typically a GaussianLikelihood or FixedNoise GaussianLikelihood.
        """
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        # constant mean as prior mean
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([bs]))
        # RBF kernel as prior covariance
        # first, specify a RBF kernel with a SmoothedBoxPrior on the lengthscale (approx Uniform)
        self.rbf = gpytorch.kernels.RBFKernel(batch_shape=torch.Size([bs]), lengthscale_prior=gpytorch.priors.SmoothedBoxPrior(1e-3, 1e+3, sigma=1e-2))
        # then, scale the RBF kernel with a SmoothedBoxPrior on the outputscale (approx Uniform)
        self.covar_module = gpytorch.kernels.ScaleKernel(self.rbf, batch_shape=torch.Size([bs]), outputscale_prior=gpytorch.priors.SmoothedBoxPrior(1e-3, 1e+3, sigma=1e-2))

    def forward(self, x):
        """A forward method that takes in some n x d data x and returns a MultivariateNormal with the prior mean and covariance evaluated at x. 
        In other words, we return the vector mu(x) and the n x n matrix representing the prior mean and covariance matrix of the GP.
        """
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
    
    
likelihood = GaussianLikelihood()

# # Instantiate the model
model = ExactGPModel(train_x, train_y, likelihood)

In [79]:
sampled_z = torch.randn((B, T, D_z))  # Input data for prediction
sampled_z = sampled_z.permute(0, 2, 1)  # Reshape to (B, D_z, T)
sampled_z = sampled_z.reshape(B * D_z, T, 1)  # Reshape to (B * D_z, T, 1)

print(f"sampled_z shape: {sampled_z.shape}")

model.eval()  # Set the model to evaluation mode
likelihood.eval()  # Set the likelihood to evaluation mode

gp_output = model(sampled_z)  # Forward pass through the model

sampled_z shape: torch.Size([128, 50, 1])


In [92]:
print(f"Caractéristiques de la sortie du modèle GP :")
print(gp_output)  # Print the output of the GP model
print(f"Mean shape: {gp_output.mean.shape}, Covariance shape: {gp_output.covariance_matrix.shape}")
print(f"Batch shape: {gp_output.batch_shape}, Event shape: {gp_output.event_shape}")

Caractéristiques de la sortie du modèle GP :
MultivariateNormal(loc: torch.Size([128, 50]), covariance_matrix: torch.Size([128, 50, 50]))
Mean shape: torch.Size([128, 50]), Covariance shape: torch.Size([128, 50, 50])
Batch shape: torch.Size([128]), Event shape: torch.Size([50])


In [96]:
mean_r = gp_output.mean.view(B, D_z, T)  # Reshape the mean to (B, D_z, T)
covar_r = gp_output.covariance_matrix.view(B, D_z, T, T)  # Reshape the covariance matrix to (B, D_z, T, T)

print(f"Mean reshaped: {mean_r.shape}, Covariance reshaped: {covar_r.shape}")

mnv_r = MultivariateNormal(
    loc=mean_r,
    covariance_matrix=covar_r
)

print(f"Caractéristiques de la distribution MVN résultante :")
print(f"Batch shape: {mvn_r.batch_shape}, Event shape: {mvn_r.event_shape}")

Mean reshaped: torch.Size([16, 8, 50]), Covariance reshaped: torch.Size([16, 8, 50, 50])


ValueError: Expected parameter covariance_matrix (Tensor of shape (16, 8, 50, 50)) of distribution MultivariateNormal(loc: torch.Size([16, 8, 50]), covariance_matrix: torch.Size([16, 8, 50, 50])) to satisfy the constraint PositiveDefinite(), but found invalid values:
tensor([[[[ 1.2246e-01,  1.3547e-01, -5.6204e-04,  ..., -1.0692e-02,
            2.2334e-03,  6.1861e-03],
          [ 1.3547e-01,  1.5537e-01, -1.8775e-04,  ..., -1.0547e-02,
            2.7847e-03,  6.7129e-03],
          [-5.6204e-04, -1.8775e-04,  1.6825e-01,  ...,  2.2143e-03,
            2.0643e-01, -1.2879e-02],
          ...,
          [-1.0692e-02, -1.0547e-02,  2.2143e-03,  ...,  3.7139e-02,
            4.9091e-03,  7.0602e-03],
          [ 2.2334e-03,  2.7847e-03,  2.0643e-01,  ...,  4.9091e-03,
            4.6627e-01, -1.6280e-02],
          [ 6.1860e-03,  6.7129e-03, -1.2879e-02,  ...,  7.0602e-03,
           -1.6280e-02,  5.7898e-02]],

         [[ 4.3903e-02, -5.2263e-03,  8.3811e-03,  ..., -1.7140e-03,
    